The objective of this notebook is to make a sample simulation of a molecule on an surfface, using one of the universal interatomic potentials available in the Open Catalyst Project. 

More information about the project can be found here:
* https://opencatalystproject.org/
* https://github.com/Open-Catalyst-Project/ocp


Packages that need to be installed (use python 3.9.*):
```
pip install ase==3.22.1 black==22.3.0 e3nn==0.4.4  numpy==1.23.5  matplotlib  numba orjson pre-commit==2.10.*  pymatgen==2023.5.10 torch==1.13.1 syrupy==3.0.6 torch_geometric==2.2.0 pyyaml tensorboard tqdm pytest   submitit  wandb lmdb torch-sparse torch-scatter

git clone https://github.com/Open-Catalyst-Project/ocp.git
cd ocp
pip install -e .
```

In ASE, the interatomic potentials are loaded as "Calculators". There are several pre-trained models that can be loaded as "Calculators".

One of them, GemNet-dT, can be downloaded from here:

```https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_08/s2ef/gemnet_t_direct_h512_all.pt``` 

This model is mentioned in the website ```https://github.com/Open-Catalyst-Project/ocp/blob/main/MODELS.md``` as part of "S2EF models: optimized for EFwT"


The calculators can be used in the following manner. Here is an example taken from:

```https://github.com/Open-Catalyst-Project/ocp/tree/main/tutorials```

In [1]:
#import the needed packages
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
# import ase.io
from ase.io.trajectory import TrajectoryReader
from ase.io import write

from ase.optimize import BFGS
from ase.build import fcc100, add_adsorbate, molecule
import os
from ase.constraints import FixAtoms
import numpy as np



In [2]:
#add a slab of fcc Cu
adslab = fcc100("Cu", size=(3, 3, 3))

#Make the molecule that will interact with the slab
adsorbate = molecule("C3H8")

#combine the slab(surface) with the molecule
add_adsorbate(adslab, adsorbate, 3, offset=(1, 1))

#fix some of the atoms
tags = np.zeros(len(adslab))
tags[18:27] = 1
tags[27:] = 2
adslab.set_tags(tags)
cons= FixAtoms(indices=[atom.index for atom in adslab if (atom.tag == 0)])
adslab.set_constraint(cons)

#add vacuum on both sides of the simulation cell
adslab.center(vacuum=13.0, axis=2)

#make the simulation cell periodic
adslab.set_pbc(True)



In [3]:
# Define the calculator
checkpoint_path = "gemnet_t_direct_h512_all.pt"
calc = OCPCalculator(checkpoint_path=checkpoint_path,
                    seed=1)

# Set up the calculator
adslab.calc = calc


In [4]:
#run a minimisation
os.makedirs("data/sample_ml_relax", exist_ok=True)
trajectory_path = "data/sample_ml_relax/toy_c3h8_relax.traj"
opt = BFGS(adslab, 
           trajectory=trajectory_path)

#Here we use a rather high value of fmax, and a small mnumber of iterations.
#fmax should be smaller, and usually more interations are needed
opt.run(fmax=0.005, steps=100)

      Step     Time          Energy         fmax
BFGS:    0 16:25:05       -4.099784        1.5675
BFGS:    1 16:25:05       -4.244462        1.1370
BFGS:    2 16:25:06       -4.403120        0.7635
BFGS:    3 16:25:07       -4.503652        0.8364
BFGS:    4 16:25:07       -4.558208        0.7339
BFGS:    5 16:25:08       -4.592069        0.4095
BFGS:    6 16:25:08       -4.619358        0.7312
BFGS:    7 16:25:09       -4.671457        0.9712
BFGS:    8 16:25:09       -4.796457        0.9211
BFGS:    9 16:25:10       -4.957972        0.9762
BFGS:   10 16:25:11       -5.109441        1.0384
BFGS:   11 16:25:11       -5.295613        1.2248
BFGS:   12 16:25:12       -5.498967        1.1272
BFGS:   13 16:25:12       -5.618080        1.0669
BFGS:   14 16:25:13       -5.737114        0.9509
BFGS:   15 16:25:13       -5.901928        0.9260
BFGS:   16 16:25:14       -6.076118        1.2737
BFGS:   17 16:25:15       -6.198387        1.2029
BFGS:   18 16:25:15       -6.250327        0.6853
B

True

In [5]:
#Save the trajectory as xyz. .xyz files can be visualised using Ovito

trajectory = TrajectoryReader(trajectory_path)
output_file_name = "data/toy_c3h8_relax.xyz"
# output_file = open(output_file_name, 'w')

for i, atoms in enumerate(trajectory):
    if i==0:
        write(output_file_name, atoms)
    else:
        write(output_file_name, atoms, append=True)


/home/sebastian/.venv/ZDDP/lib/python3.9/site-packages/ase/io/extxyz.py:302: UserWarning: Skipping unhashable information adsorbate_info
  warnings.warn('Skipping unhashable information '
